In [0]:
%sql
create table if not exists duebu_datamodelling.silver.silver_data (
  order_id int primary key,
  order_date date,
  customer_id int,
  customer_name varchar(200),
  customer_email varchar(100),
  product_id int,
  product_name varchar(100),
  product_category varchar(50),
  quantity int,
  unit_price decimal(10,2),
  payment_type varchar(50),
  country varchar(50),
  last_updated date
)

In [0]:
%sql
create or replace view tv_silver as
select * from duebu_datamodelling.bronze.bronze_data
order by customer_id

In [0]:
%sql
WITH deduplicated_source AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updated DESC) AS row_num
    FROM tv_silver
    
)
MERGE INTO duebu_datamodelling.silver.silver_data st
USING (select * from deduplicated_source where row_num = 1) sv
ON st.customer_id = sv.customer_id
WHEN MATCHED THEN 
    UPDATE SET 
        st.customer_id = sv.customer_id,
        st.customer_name = sv.customer_name,
        st.customer_email = sv.customer_email,
        st.country = sv.country,
        st.last_updated = sv.last_updated


In [0]:
%sql
WITH deduplicated_source AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY last_updated DESC) AS row_num
    FROM tv_silver
)
MERGE INTO duebu_datamodelling.silver.silver_data st
USING (SELECT * FROM deduplicated_source WHERE row_num = 1) sv
ON st.order_id = sv.order_id
WHEN MATCHED THEN 
    UPDATE SET 
        st.order_id = sv.order_id,
        st.order_date = sv.order_date,
        st.product_id = sv.product_id,
        st.product_name = sv.product_name,
        st.product_category = sv.product_category,
        st.quantity = sv.quantity,
        st.unit_price = sv.unit_price,
        st.payment_type = sv.payment_type,
        st.last_updated = sv.last_updated
WHEN NOT MATCHED THEN 
    INSERT (
        order_id, order_date, customer_id, customer_name, customer_email, 
        product_id, product_name, product_category, quantity, unit_price, 
        payment_type, country, last_updated
    ) 
    VALUES (
        sv.order_id, sv.order_date, sv.customer_id, sv.customer_name, sv.customer_email, 
        sv.product_id, sv.product_name, sv.product_category, sv.quantity, sv.unit_price, 
        sv.payment_type, sv.country, sv.last_updated
    )

In [0]:
%sql
select max(order_date) ,count(*)
from duebu_datamodelling.silver.silver_data

In [0]:
%sql
select * from duebu_datamodelling.silver.silver_data where order_id=10102

In [0]:
%sql
CREATE FUNCTION ssn_mask(ssn STRING)
  RETURN CASE WHEN is_account_group_member('HumanResourceDept') THEN ssn ELSE '***-**-****' END;

ALTER TABLE <table_name> ALTER COLUMN <col_name> SET MASK <mask_func_name> USING COLUMNS <additional_columns>;

In [0]:
%sql
drop function duebu_datamodelling.silver.data_mask_name;
drop function duebu_datamodelling.silver.data_mask_email

In [0]:
%sql
create or replace function duebu_datamodelling.silver.data_mask_name(p_customer_name string)
    return case when is_account_group_member('data_mask') then p_customer_name else '***** ******' 
end;

create or replace function duebu_datamodelling.silver.data_mask_email(p_customer_email string)
    return case when is_account_group_member('data_mask') then p_customer_email else '*****@****.***' 
end;


In [0]:
%sql
alter table duebu_datamodelling.silver.silver_data 
alter column customer_name 
set mask duebu_datamodelling.silver.data_mask_name

In [0]:
%sql
alter table duebu_datamodelling.silver.silver_data 
alter column customer_email
set mask duebu_datamodelling.silver.data_mask_email

In [0]:
%sql
select * from duebu_datamodelling.silver.silver_data